In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from abelfunctions import RiemannTheta

import layers as layers
import model as mdl

import warnings
warnings.filterwarnings('ignore')

from costfunctions import mse, crossentropy
import minimizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer

# Example 1: Sine-Mixture with gaussian noise

In [ ]:
def func(x,n):
    return 0.5*np.sin(1*x+0.1)+0.75*np.sin(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(1,3))
M.add(layers.ThetaUnitLayer(3,3))
M.add(layers.ThetaUnitLayer(3,1))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-5)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 2: Sine-Cosine-Mixture with trend and gaussian noise

In [ ]:
def func(x,n):
    return 0.02*x+0.5*np.sin(1*x+0.1)+0.75*np.cos(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
M = mdl.Model()

M.add(layers.ThetaUnitLayer(1,3, param_bound=10))
M.add(layers.ThetaUnitLayer(3,3, param_bound=10))
M.add(layers.ThetaUnitLayer(3,2, param_bound=10))
M.add(layers.ThetaUnitLayer(2,1, param_bound=10))

In [ ]:
minim = minimizer.CMA(parallel=True)
minim.train(mse, M, X, Y, tolfun=1e-5)

In [ ]:
npoints = 500
test_X = (np.linspace(0, 100, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Example 3: MNIST (work in progress...)

In [ ]:
# Load MNIST dataset
MNIST_train = pd.read_csv('~/data/mnist_train.csv', delimiter=",",header=None).values
MNIST_test  = pd.read_csv('~/data/mnist_test.csv', delimiter=",",header=None).values

# Prepare data (normalized onto [0,1])
Y_train = MNIST_train[0:100,0]
X_train = MNIST_train[0:100,1:]/255.0

Y_test = MNIST_test[:,0]
X_test = MNIST_test[:,1:]/255.0

In [ ]:
M = mdl.Model()
M.add(layers.ThetaUnitLayer(784,10))
M.add(layers.SoftMaxLayer(10))
M.add(layers.MaxPosLayer(10,1))

#enc = LabelBinarizer()
#enc.fit(np.diag([1,1,1,1,1,1,1,1,1,1]))
#enc.classes_ = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#T=enc.transform(Y_train)

# ToDo: Use cross-entropy

In [ ]:
minim = minimizer.CMA(True)

minim.train(crossentropy,M, np.transpose(X_train), np.transpose(Y_train), maxiter=5)

In [ ]:
W=M.predict(np.transpose(X_train))

In [ ]:
W

In [ ]:
crossentropy(W,np.transpose(T))

In [ ]:
P=np.real(M.predict(np.transpose(X_test)))

#where_are_NaNs = np.isnan(P)
#P[where_are_NaNs] = 0
#P = np.abs(np.round(P))
P=P.reshape(P.shape[0],)

print(classification_report(Y_test,P))
print(confusion_matrix(Y_test, P))